#### Demo Notebook
- Can play with pretrianed files locally

In [1]:
! make clean

rm -rf dist
find . -type f -name "*.DS_Store" -ls -delete
find . | grep -E "(__pycache__|\.pyc|\.pyo)" | xargs rm -rf
find . | grep -E ".pytest_cache" | xargs rm -rf
find . | grep -E ".ipynb_checkpoints" | xargs rm -rf
rm -f .coverage


In [2]:
! make clean-logs

rm -rf logs/**
rm -rf wandb/**


In [3]:
# ! rm  -rf /media/sayem/510B93E12554BBD1/Hangman/wandb
# ! rm -rf /media/sayem/510B93E12554BBD1/checkpoints 

In [4]:
import torch
import numpy as np
import random

def set_seed(seed):
    """Set seed for reproducibility."""
    random.seed(seed)       # Python random module
    np.random.seed(seed)    # Numpy module
    torch.manual_seed(seed) # PyTorch
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)          # Sets seed for CUDA (GPU)
        torch.cuda.manual_seed_all(seed)      # Ensure reproducibility on all GPUs
        torch.backends.cudnn.deterministic = True  # Use deterministic algorithms
        torch.backends.cudnn.benchmark = False     # If input sizes do not vary, this should be set to False

# Example usage: 
set_seed(42)  # Use any number to seed all libraries

In [5]:
from src.utils import read_word_list, read_rnd_word_list
from src.data_generation.dataset_analysis import stratified_sample_from_categories, \
    classify_words_by_unique_letter_count, summarize_categories, categorize_words_by_length
corpus_path = 'data/words_250000_train.txt' # ## Just a demo, replace with your corpus path
total_samples = 250_000  # This can be adjusted as needed

# Read a specified number of words from the corpus
corpus = read_rnd_word_list(corpus_path, num_samples=total_samples)

# classification = categorize_words_by_length(corpus)

# summaries = summarize_categories(classification)

# corpus = classification[2]
# print(len(corpus))

In [6]:
# # from src.datamodule import HangmanDataset
# from src.datamodule import HangmanDataModule
import gymnasium as gym
torch.cuda.empty_cache()

# torch.set_float32_matmul_precision('high')
import lightning as L 
# # L.seed_everything(102, workers=True)
# np.random.seed(102)  # You can use any number here

##### Env

In [7]:
from gymnasium.envs.registration import register

# Registration code without default kwargs
register(
    id='Hangman-v0',  # Use an environment ID that follows Gym conventions
    entry_point='rl_squared.envs.hangman.hangman_env:HangmanEnv',  # Module and class name
)

In [8]:
from tqdm import tqdm
from rl_squared.utils.env_utils import make_vec_envs, make_env_thunk
from rl_squared.training.meta_episode_batch import MetaEpisodeBatch
from rl_squared.envs.hangman.hangman_env import HangmanEnv
import numpy as np
from gymnasium.envs.registration import register

from rl_squared.networks.stateful.stateful_actor_critic \
                import StatefulActorCritic

from rl_squared.envs.hangman.hangman_env import HangmanEnv
import numpy as np
import copy
from rl_squared.networks.modules.distributions import MaskableCategoricalDistribution
config = {'word_list': corpus, 'max_attempts': 6, \
        'max_length': 35, 'auto_reset': True,}


total_samples = 250_000  # This can be adjusted as needed

corpus = read_word_list(corpus_path, num_samples=total_samples)
num_meta_episodes = len(corpus)
print(f"num_meta_episodes = {num_meta_episodes}")
num_parallel_envs = 3

# Assuming make_vec_envs is a function defined elsewhere that creates vectorized environments
rl_squared_envs = make_vec_envs('Hangman-v0', \
                config, 200, num_parallel_envs, 'cuda')

num_meta_episodes = 14


In [9]:
from rl_squared.networks.stateful.stateful_actor_critic import StatefulActorCritic
from rl_squared.networks.stateful.stateful_actor import StatefulActor
from rl_squared.networks.stateful.stateful_critic import StatefulCritic

actor_critic = StatefulActorCritic(
        rl_squared_envs.observation_space,
        rl_squared_envs.action_space,
        recurrent_state_size=1024,
).to_device('cuda')

# actor = StatefulActor(
#         observation_space=rl_squared_envs.observation_space,
#         action_space=rl_squared_envs.action_space,
#         recurrent_state_size=512,
#         hidden_sizes=[256],
# )

# x, recurrent_state = actor(observations, )

In [10]:
ckt_path = "results/hangman-v0/run-1725320963/checkpoints/checkpoint-best.pt"

checkpoint = torch.load(ckt_path, map_location='cpu', \
                    weights_only=True)
# current_iteration = checkpoint["iteration"]
# actor_critic.actor.load_state_dict(checkpoint["actor"])
# actor_critic.critic.load_state_dict(checkpoint["critic"])
actor_critic.load_state_dict(checkpoint["actor_critic"])

<All keys matched successfully>

In [11]:
# import torch
# import os
# import glob
# from collections import OrderedDict

# # Define the base directory and pattern for checkpoint files
# base_dir = "./_results/hangman-v0"
# pattern = "**/*-best.pt"
# search_pattern = os.path.join(base_dir, pattern)

# # Find all matching checkpoint files
# checkpoint_files = glob.glob(search_pattern, recursive=True)

# # Load all the checkpoints with weights_only=True for safety
# checkpoints = [torch.load(path, map_location='cpu', \
#                           weights_only=True) for path in checkpoint_files]

# def average_checkpoints(checkpoint_list):
#     """Average the state dictionaries from a list of loaded checkpoints."""
#     avg_state_dict = OrderedDict()
#     # Initialize the average state dictionary with zeros
#     for key in checkpoint_list[0]['actor_critic'].keys():
#         avg_state_dict[key] = torch.zeros_like(checkpoint_list[0]['actor_critic'][key])
#     # Sum all parameters
#     for checkpoint in checkpoint_list:
#         for key, value in checkpoint['actor_critic'].items():
#             avg_state_dict[key] += value
#     # Average the parameters
#     for key in avg_state_dict.keys():
#         avg_state_dict[key] /= len(checkpoint_list)
#     return avg_state_dict

# # Perform averaging if checkpoints are loaded
# if checkpoints:
#     avg_state_dict = average_checkpoints(checkpoints)
#     # Load the averaged state dictionary into the pre-defined actor-critic model
#     actor_critic.load_state_dict(avg_state_dict)
#     print("Averaged model parameters loaded into the actor-critic model.")
# else:
#     print("No checkpoints were found or loaded.")

In [12]:
from src.data_generation.simulation import play_a_game_with_a_word, \
    simulate_games_for_word_list # testing function
# from src.data_generation.process_word import process_word
# from src.datamodule.dataset import encode_character
from src.model.inference import guess #, guess_character
from src.data_generation.simulation import play_a_game_with_a_word, \
                            simulate_games_for_word_list # testing function
from src.data_generation.data_generation \
    import simulated_guess_function, generate_a_game_with_a_word

# solver = HangmanFreqSolver(corpus) # TODO: does it matter waht corpus, 
# since no use use in guess?
# transform = ProcessWordTransform(corpus) # here, what corpus does not matter

# Example word
# word = 'jazz' 
# word = 'superacknowledgment'
word = 'mississippi' # out of corpus
# word = 'embryoplastic' # from corpus
# word = "aandahl"
masked_word = '_' * len(word)
# masked_word = ''
guessed_letters = []

In [13]:
['pulmobranchiate', 'senzer', 'quasicomprehensively', 'embryoplastic']

['pulmobranchiate', 'senzer', 'quasicomprehensively', 'embryoplastic']

In [14]:
actor_critic.recurrent_state_size

1024

In [15]:
gussed_character, recurrent_states_actor, recurrent_states_critic \
    = guess(model=actor_critic, 
            word=masked_word, # TODO
            guessed_letters=guessed_letters,
            previous_action='a',
            recurrent_states_actor=None,
            recurrent_states_critic=None)

print(gussed_character) 

i


In [16]:
play_a_game_with_a_word(word=word, \
            guess_function=guess, model=actor_critic) # aggregated_data=None): # TODO: aggregated_data=None: remove later

(False,
 0,
 [('t', '___________', False),
  ('e', '___________', False),
  ('i', '_i__i__i__i', True),
  ('l', '_i__i__i__i', False),
  ('c', '_i__i__i__i', False),
  ('n', '_i__i__i__i', False),
  ('s', '_ississi__i', True),
  ('r', '_ississi__i', False)])

In [17]:
# (False,
#  0,
#  [('e', '____', False),
#   ('a', '_a__', True),
#   ('s', '_a__', False),
#   ('n', '_a__', False),
#   ('t', '_a__', False),
#   ('r', '_a__', False),
#   ('l', '_a__', False)])

In [18]:
import random

def read_rnd_word_list(file_path, num_samples=1_000):
    with open(file_path, 'r') as file:
        words = file.readlines()  # Read all lines from the file
        words = [word.strip() for word in words]  # Remove any extra whitespace or newline characters

    # Shuffle the list of words to randomize the order
    random.shuffle(words)

    # Return the first num_samples words if the list is longer than num_samples
    return words[:num_samples] if len(words) > num_samples else words

# corpus_path_ = '/media/sayem/510B93E12554BBD1/Hangman/data/hidden_words.txt'
# selected_words = read_rnd_word_list(corpus_path_, num_samples=110)

In [19]:

corpus_path_ = 'data/words_250000_train.txt' ## Just a demo, replace with your corpus path

# word_list = read_word_list(corpus_path_, num_samples=110)

from src.data_generation.simulation import play_a_game_with_a_word, \
                            simulate_games_for_word_list # testing function

# for _ in range(10):
selected_words = read_rnd_word_list(corpus_path_, num_samples=250_000)
final_results = simulate_games_for_word_list(word_list=selected_words, guess_function=guess, \
                                            play_function=play_a_game_with_a_word, \
                                            model=actor_critic) 

# Print overall statistics
overall_stats = final_results['overall']
print("\nOverall Statistics:") 
print(f"Total Games: {overall_stats['total_games']}, Wins: {overall_stats['wins']}, Losses: {overall_stats['losses']}")
print(f"Win Rate: {overall_stats['win_rate']:.2f}, Average_tries_remaining: {overall_stats['average_tries_remaining']:.2f}")
print()

Simulating games: 100%|██████████| 14/14 [00:00<00:00, 63.26word/s]


Overall Statistics:
Total Games: 14, Wins: 7, Losses: 7
Win Rate: 0.50, Average_tries_remaining: 1.64



In [20]:
STOP

NameError: name 'STOP' is not defined

#### API Integration

In [ ]:
from src.api import HangmanAPI
import time

In [ ]:
# Specify the path to the file
file_path = 'notebooks/api_key.txt'

# Use a context manager to open and read the file
with open(file_path, 'r') as file:
    api_key = file.read().strip()  # Read the content and strip any extra whitespace

# print("API Key:", api_key)

In [ ]:
api = HangmanAPI(model=actor_critic, corpus_path=corpus_path, access_token=api_key, timeout=2_000)

In [ ]:
api.start_game(practice=1, verbose=True)
[total_practice_runs, total_recorded_runs,total_recorded_successes, total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

In [ ]:
prev =[total_practice_runs, total_recorded_runs, \
            total_recorded_successes, total_practice_successes] = api.my_status()

In [ ]:
[total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] 

In [ ]:
# for i in tqdm(range(1_000)):
#     # print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     # api.start_game(practice=0,verbose=False)
#     api.start_game(practice=1, verbose=False)
#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

In [ ]:
current =[total_practice_runs, total_recorded_runs, \
            total_recorded_successes, total_practice_successes] = api.my_status()

## Playing recorded games:

In [ ]:
for i in tqdm(range(1000)):
    # print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)